In [17]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv("imdb_master.csv",encoding="ISO-8859-1")

In [3]:
df=df.replace("neg", 0)
df=df.replace("pos", 1)
df.head()

,Unnamed: 0,type,review,label,file
0,0,test,Once again Mr. Costner has dragged out a movie...,0,0_2.txt
1,1,test,This is an example of why the majority of acti...,0,10000_4.txt
2,2,test,"First of all I hate those moronic rappers, who...",0,10001_1.txt
3,3,test,Not even the Beatles could write songs everyon...,0,10002_3.txt
4,4,test,Brass pictures (movies is not a fitting word f...,0,10003_3.txt


In [4]:
traindf=df[df["type"]=="train"]
testdf=df[df["type"]=="test"]
traindf=traindf[["review","label"]]
testdf=testdf[["review","label"]]
print(len(traindf))
print(len(testdf))

75000
25000


In [5]:
X_train=traindf["review"].values
y_train=traindf["label"].values
X_test=testdf["review"].values
y_test=testdf["label"].values

In [20]:
X_train[0:5]

array(["Story of a man who has unnatural feelings for a pig. Starts out with a opening scene that is a terrific example of absurd comedy. A formal orchestra audience is turned into an insane, violent mob by the crazy chantings of it's singers. Unfortunately it stays absurd the WHOLE time with no general narrative eventually making it just too off putting. Even those from the era should be turned off. The cryptic dialogue would make Shakespeare seem easy to a third grader. On a technical level it's better than you might think with some good cinematography by future great Vilmos Zsigmond. Future stars Sally Kirkland and Frederic Forrest can be seen briefly.",
       "Airport '77 starts as a brand new luxury 747 plane is loaded up with valuable paintings & such belonging to rich businessman Philip Stevens (James Stewart) who is flying them & a bunch of VIP's to his estate in preparation of it being opened to the public as a museum, also on board is Stevens daughter Julie (Kathleen Quinlan

In [21]:
import tensorflow as tf
import keras
from keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.preprocessing.text import Tokenizer

tokenizer_obj=Tokenizer()
total_reviews=X_train
tokenizer_obj.fit_on_texts(total_reviews)

max_length=max([len(s.split()) for s in total_reviews])

vocab_size=len(tokenizer_obj.word_index)+1
X_train_tokens=tokenizer_obj.texts_to_sequences(X_train)
X_test_tokens=tokenizer_obj.texts_to_sequences(X_test)

X_train_pad=pad_sequences(X_train_tokens,maxlen=max_length,padding="post")
X_test_pad=pad_sequences(X_test_tokens,maxlen=max_length,padding="post")


In [22]:
from keras import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout,CuDNNLSTM,Bidirectional,TimeDistributed,Activation,Flatten
EMB_DIM=300
model=Sequential()
model.add(Embedding(vocab_size, EMB_DIM, input_length=max_length))
model.add(Bidirectional(CuDNNLSTM(256)))
model.add(Dropout(0.4))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 2470, 300)         45867600  
_________________________________________________________________
bidirectional_2 (Bidirection (None, 512)               1142784   
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 513       
Total params: 47,010,897
Trainable params: 47,010,897
Non-trainable params: 0
_________________________________________________________________
None


In [24]:
batch_size = 256
num_epochs = 10
model.fit(X_train_pad, y_train, validation_split=0.2, batch_size=batch_size, epochs=num_epochs)

Train on 60000 samples, validate on 15000 samples
Epoch 1/10


ValueError: could not convert string to float: 'unsup'

In [ ]:
scores = model.evaluate(X_test_pad, y_test, verbose=0)
print('Test accuracy:', scores[1])
model.save("model.h5")